In [ ]:
!sudo apt-get install git-lfs
!git lfs install

In [24]:
!git clone https://huggingface.co/Skoltech/russian-sensitive-topics

Cloning into 'russian-sensitive-topics'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 36 (delta 14), reused 0 (delta 0)
Unpacking objects: 100% (36/36), done.
Filtering content: 100% (4/4), 1.99 GiB | 55.57 MiB/s, done.


In [25]:
from graph import InferenceWorker
import torch
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('/content/russian-sensitive-topics')
model = AutoModelForSequenceClassification.from_pretrained('/content/russian-sensitive-topics', return_dict=True)

In [26]:
text = 'В стране карантин. Умерло очень много людей.'

In [27]:
text1 = 'ЦИК Туркменистана сообщил, что досрочные выборы президента назначены на 12 марта. Накануне вечером действующий глава государства Гурбангулы Бердымухамедов заявил, что принял "непростое решение о себе" и что путь во властные структуры следует предоставить молодым руководителям.'

In [39]:
class SampleSensitive(InferenceWorker):
    def __call__(self, t, d):
        if t == "text":
            yield "sensitive", self.predict(d), "Sensitive_topics"
        else:
            yield None        
    @torch.no_grad()
    def predict(self, text):
        inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
        outputs = model(**inputs)
        predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
        predicted = torch.argmax(predicted, dim=1).numpy()
        if predicted[0] == 1:
          return 'offline_crime'
        if predicted[0] == 2:
          return 'online_crime'
        if predicted[0] == 3:
          return 'drugs'   
        if predicted[0] == 4:
          return 'gambling'
        if predicted[0] == 5:
          return 'pornography'
        if predicted[0] == 6:
          return 'prostitution'  
        if predicted[0] == 7:
          return 'slavery'
        if predicted[0] == 8:
          return 'suicide'
        if predicted[0] == 9:
          return 'terrorism'  
        if predicted[0] == 10:
          return 'weapons'
        if predicted[0] == 11:
          return 'body_shaming'
        if predicted[0] == 12:
          return 'health_shaming'
        if predicted[0] == 13:
          return 'politics' 
        if predicted[0] == 14:
          return 'racism'
        if predicted[0] == 15:
          return 'religion'   
        if predicted[0] == 16:
          return 'sexual_minorities'
        if predicted[0] == 17:
          return 'sexism'
        if predicted[0] == 18:
          return 'social_injustice' 


In [40]:
worker = SampleSensitive()
worker.serialize()

Worker saved!


In [30]:
for x in worker("text", text):
  print(x)

('sensitive', 'offline_crime', 'Sensitive_topics')


In [31]:
for x in worker("text", text1):
  print(x)

('sensitive', 'racism', 'Sensitive_topics')
